In [1]:
# [ 토큰값 및 경로 설정 ]
from pathlib import Path
import os, sys

# ---- 프로젝트 루트 & sys.path 설정 ----
PROJECT_ROOT = Path("/workspace/baseball_pipeline").resolve()
os.chdir(PROJECT_ROOT)
sys.path.append(str(PROJECT_ROOT))  # src 패키지 임포트용

print("PROJECT_ROOT:", PROJECT_ROOT)

# ---- 데이터 디렉토리 ----
DATA_DIR = PROJECT_ROOT / "data"
INPUT_VIDEO_DIR = DATA_DIR / "input_videos"
STT_RAW_DIR = DATA_DIR / "stt_raw"
STT_SEG_DIR = DATA_DIR / "stt_segments"
LLM_OUT_DIR = DATA_DIR / "llm_outputs"
FAISS_DIR = PROJECT_ROOT / "faiss_index"
SRC_ROOT = PROJECT_ROOT / "src"


if str(SRC_ROOT) not in sys.path:
    sys.path.append(str(SRC_ROOT))

print("sys.path 추가됨:", SRC_ROOT)


for d in (DATA_DIR, INPUT_VIDEO_DIR, STT_RAW_DIR, STT_SEG_DIR, LLM_OUT_DIR, FAISS_DIR, SRC_ROOT):
    d.mkdir(parents=True, exist_ok=True)

print("INPUT_VIDEO_DIR:", INPUT_VIDEO_DIR) 
print("STT_RAW_DIR    :", STT_RAW_DIR)
print("STT_SEG_DIR    :", STT_SEG_DIR)
print("LLM_OUT_DIR :", LLM_OUT_DIR)
print("FAISS_DIR      :", FAISS_DIR)
print("SRC_ROOT      :", SRC_ROOT)

# ---- CLOVA 토큰 (하드코딩) ----
CLOVA_INVOKE_URL = ""  # 네 invoke URL
CLOVA_SECRET_KEY = ""                                       # 네 secret

# (선택) 다른 토큰들
HF_TOKEN = ""
OPENAI_API_KEY = ""

if HF_TOKEN and "xxx" not in HF_TOKEN:
    os.environ["HF_TOKEN"] = HF_TOKEN
    os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
    os.environ["HUGGING_FACE_HUB_TOKEN"] = HF_TOKEN

if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ 기본 경로/토큰 설정 완료")

# 20초

PROJECT_ROOT: /workspace/baseball_pipeline
sys.path 추가됨: /workspace/baseball_pipeline/src
INPUT_VIDEO_DIR: /workspace/baseball_pipeline/data/input_videos
STT_RAW_DIR    : /workspace/baseball_pipeline/data/stt_raw
STT_SEG_DIR    : /workspace/baseball_pipeline/data/stt_segments
LLM_OUT_DIR : /workspace/baseball_pipeline/data/llm_outputs
FAISS_DIR      : /workspace/baseball_pipeline/faiss_index
SRC_ROOT      : /workspace/baseball_pipeline/src
✅ 기본 경로/토큰 설정 완료


In [4]:
# 구글 드라이브 설치
!pip install gdown

In [21]:
# [셀: 구글 드라이브 링크로 영상 다운로드 → INPUT_VIDEO_DIR에 저장]

!pip install -q gdown

import re
from pathlib import Path

import gdown
from src import DATA_DIR  # 이건 src/__init__.py 에 이미 있는 거라 import 가능

# 여기서 INPUT_VIDEO_DIR 다시 정의
INPUT_VIDEO_DIR = DATA_DIR / "input_videos"
INPUT_VIDEO_DIR.mkdir(parents=True, exist_ok=True)

def download_gdrive_video(gdrive_url: str, dest_name: str | None = None) -> Path:
    """
    구글 드라이브 공유 링크(gdrive_url)를 받아서
    INPUT_VIDEO_DIR 아래에 dest_name 이름으로 저장.

    - gdrive_url: 드라이브 '공유 링크'
    - dest_name: 저장할 파일 이름 (예: "clip.mp4")
                 None이면 file id 기반 이름으로 저장
    """
    # 1) 링크에서 file id 추출
    m = re.search(r"id=([a-zA-Z0-9_-]+)", gdrive_url)
    if not m:
        m = re.search(r"/d/([a-zA-Z0-9_-]+)/", gdrive_url)

    if not m:
        raise ValueError(
            f"구글 드라이브 링크에서 file id를 찾지 못했어.\n"
            f"현재 url: {gdrive_url}"
        )

    file_id = m.group(1)

    # 2) 저장 파일 이름 결정
    if dest_name is None:
        dest_name = f"{file_id}.mp4"

    out_path = INPUT_VIDEO_DIR / dest_name

    # 3) gdown용 다운로드 URL 구성
    url = f"https://drive.google.com/uc?id={file_id}"
    print("[GDRIVE] file_id:", file_id)
    print("[GDRIVE] url    :", url)
    print("[GDRIVE] output :", out_path)

    gdown.download(url, str(out_path), quiet=False)

    print("[GDRIVE] 다운로드 완료:", out_path, "| exists:", out_path.exists())
    return out_path


# ====== 여기만 바꿔서 쓰면 됨 ======
# 1) 구글 드라이브에서 '공유 링크' 복사해서 아래에 붙여넣기
#gdrive_url = "https://drive.google.com/file/d/1_iMy7rbt44vvxO-uOcxsmBuKRszH-mrR/view?usp=sharing"
gdrive_url = "https://drive.google.com/file/d/1Xnq5p4SQCtREkh3Re_FaCexTjpm7JjLi/view?usp=sharing"

# 2) Runpod에서 사용할 파일 이름 (파이프라인 기준 clip.mp4)
local_video_path = download_gdrive_video(gdrive_url, dest_name="clip_test4.mp4")
local_video_path

# 40초

[GDRIVE] file_id: 1Xnq5p4SQCtREkh3Re_FaCexTjpm7JjLi
[GDRIVE] url    : https://drive.google.com/uc?id=1Xnq5p4SQCtREkh3Re_FaCexTjpm7JjLi
[GDRIVE] output : /workspace/baseball_pipeline/data/input_videos/clip_test4.mp4


Downloading...
From: https://drive.google.com/uc?id=1Xnq5p4SQCtREkh3Re_FaCexTjpm7JjLi
To: /workspace/baseball_pipeline/data/input_videos/clip_test4.mp4
100%|██████████| 92.5M/92.5M [00:01<00:00, 76.7MB/s]


[GDRIVE] 다운로드 완료: /workspace/baseball_pipeline/data/input_videos/clip_test4.mp4 | exists: True


PosixPath('/workspace/baseball_pipeline/data/input_videos/clip_test4.mp4')

In [22]:
# [ 네이버 Speech STT 모델 API를 통한 텍스트 요청 ]

from src.stt_pipeline import run_stt_pipeline

VIDEO_NAME = "clip_test4.mp4"
video_path = INPUT_VIDEO_DIR / VIDEO_NAME

print("video_path:", video_path, "| exists:", video_path.exists())
if not video_path.exists():
    raise FileNotFoundError("data/input_videos/ 안에 clip.mp4 를 넣어주거나 VIDEO_NAME을 변경해줘.")

# (선택) 키워드 엑셀
STT_KEYWORD_XLSX = PROJECT_ROOT / "stt.xlsx"
if STT_KEYWORD_XLSX.exists():
    xlsx_path = STT_KEYWORD_XLSX
    use_domain = False
    print("[STT] stt.xlsx 사용 (엑셀 부스팅)")
else:
    xlsx_path = None
    use_domain = True
    print("[STT] stt.xlsx 없음 → 도메인 키워드 부스팅만 사용")

tts_csv_path, timeline_json_path = run_stt_pipeline(
    audio_path=video_path,
    invoke_url=CLOVA_INVOKE_URL,
    secret_key=CLOVA_SECRET_KEY,
    stt_raw_dir=STT_RAW_DIR,
    stt_seg_dir=STT_SEG_DIR,
    xlsx_keywords_path=xlsx_path,
    use_domain_boostings=use_domain,
    speaker_count_min=2,
    speaker_count_max=3,
    save_raw_json=True,
)

tts_csv_path, timeline_json_path

# 120초

video_path: /workspace/baseball_pipeline/data/input_videos/clip_test4.mp4 | exists: True
[STT] stt.xlsx 사용 (엑셀 부스팅)
[STT_PIPELINE] Clova STT 요청 시작: /workspace/baseball_pipeline/data/input_videos/clip_test4.mp4
[STT_PIPELINE] raw JSON 저장 -> /workspace/baseball_pipeline/data/stt_raw/clip_test4.clova_raw.json
[STT_PIPELINE] phrase CSV 저장 -> /workspace/baseball_pipeline/data/stt_segments/clip_test4.tts_phrases.csv (rows=8)
[STT_PIPELINE] timeline JSON 저장 -> /workspace/baseball_pipeline/data/stt_segments/clip_test4.timeline.json


(PosixPath('/workspace/baseball_pipeline/data/stt_segments/clip_test4.tts_phrases.csv'),
 PosixPath('/workspace/baseball_pipeline/data/stt_segments/clip_test4.timeline.json'))

In [8]:
# [ STT 데이터 → 박찬호 LLM 변환 단계 (심플 버전) ]

from pathlib import Path
from src import DATA_DIR

from src.llm_kanana import run_llm_kanana_for_tts_csv

STT_SEG_DIR = DATA_DIR / "stt_segments"
LLM_OUT_DIR = DATA_DIR / "llm_outputs"

VIDEO_NAME = "clip_test1.mp4"
VIDEO_STEM = Path(VIDEO_NAME).stem

TTS_CSV = STT_SEG_DIR / f"{VIDEO_STEM}.tts_phrases.csv"
OUT_CSV = LLM_OUT_DIR / f"{VIDEO_STEM}.tts_phrases.llm_kanana.csv"

#GAME_DESC = "2025 준플레이오프 2차전"
GAME_DESC = "2015 WBSC 프리미어12 준결승 2경기 대한민국 vs 일본"

print("입력 CSV :", TTS_CSV, "| exists:", TTS_CSV.exists())
print("출력 CSV :", OUT_CSV)

out_csv = run_llm_kanana_for_tts_csv(
    csv_path=TTS_CSV,
    out_path=OUT_CSV,
    game_desc=GAME_DESC,
)

out_csv



/workspace/miniconda3/envs/baseball_pipe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


입력 CSV : /workspace/baseball_pipeline/data/stt_segments/clip_test1.tts_phrases.csv | exists: True
출력 CSV : /workspace/baseball_pipeline/data/llm_outputs/clip_test1.tts_phrases.llm_kanana.csv
[KANANA] loading base model on cuda ...


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:59<00:00, 14.92s/it]


[KANANA] loading LoRA: SeHee8546/kanana-1.5-8b-pakchanho-lora
[KANANA] model + LoRA ready.


/workspace/baseball_pipeline/src/llm_kanana.py:283: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '예, 아, 박찬호 위원입니다. 네, 예, 투 스트라이크, 투 볼, 5번 타자, 일본의 후쿠하라가 지금 타석에 들어섰습니다. 예, 투 스트라이크, 투 볼, 이, 예, 지금 투 스트라이크가 된 공은, 예, 지금 포크볼이었고요. 예, 자, 바깥쪽, 낮은, 예, 지금 포크볼, 예, 지금은 지금 몸 쪽, 지금은 바깥쪽, 예, 높게, 예, 자, 예, 예, 지금은' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "llm_text"] = reply


[KANANA] idx=0 utt=clip_1_1 role=analyst orig='최은성 5번...' -> llm='예, 아, 박찬호 위원입니다. 네, 예, 투 스트라이크...'
[KANANA] idx=1 utt=clip_2_1 role=analyst orig='자 초구를 당겼고 이 타구...' -> llm='네, 그렇죠. 초구를 당겼고, 이 타구가 지금 우측으로...'
[KANANA] idx=2 utt=clip_2_2 role=analyst orig='3유간을 뺐습니다. 플로리얼 달립니다. 홈까지 홈까지 ...' -> llm='네, 박찬호 위원입니다....'
[KANANA] idx=3 utt=clip_3_1 role=analyst orig='채은성의 적시타...' -> llm='네, 그렇죠. 채은성 선수가 좌측, 3루 쪽으로 적시타...'
[KANANA] idx=4 utt=clip_3_2 role=analyst orig='스코어 2 대 0. 한화 이글스...' -> llm='네, 지금 1루 주자 3루에 진루해 있고, 3루 주자가...'
[KANANA] idx=10 utt=clip_6_1 role=analyst orig='문현빈의 희생 플라이, 그리고 채은성의 24 타점...' -> llm='네, 문현빈의 희생 플라이로 1루 주자가 2루까지, 그...'
[KANANA] idx=11 utt=clip_6_2 role=analyst orig='기민한 주자...' -> llm='예, 저, 지금, 대한민국, 대한민국의, 아, 주자, ...'
[KANANA] idx=12 utt=clip_6_3 role=analyst orig='플로우 리얼리...' -> llm='자, 플로우 리얼리. 투수가 지금 이 순간에 어떤 공을...'
[KANANA] idx=13 utt=clip_7_1 role=analyst orig='홈까지...' -> llm='네, 저희 투수가 투 스트라이크까지 잘 잡아내고 있거든...'
[KANANA] idx=14 utt=clip_7_2 role=analys

PosixPath('/workspace/baseball_pipeline/data/llm_outputs/clip_test1.tts_phrases.llm_kanana.csv')

In [ ]:
# [ Fish-Speech api_server 기반 TTS 생성 ]
from pathlib import Path

from src import DATA_DIR
from src.tts_fishspeech_api import run_tts_batch_via_api

LLM_OUT_DIR = DATA_DIR / "llm_outputs"
TTS_REF_DIR = DATA_DIR / "tts_refs"

# VIDEO_NAME = "clip.mp4"
VIDEO_NAME = "clip_test4.mp4"
VIDEO_STEM = Path(VIDEO_NAME).stem

llm_csv = LLM_OUT_DIR / f"{VIDEO_STEM}.tts_phrases.llm_kanana.csv"

# 캐스터 참조 음성 후보들 (있는 것만 사용)
caster_ref_wavs = [
    TTS_REF_DIR / "caster_prompt_1.wav",
    # TTS_REF_DIR / "caster_prompt_2.wav",
    # TTS_REF_DIR / "caster_prompt_3.wav",
]

# 해설위원 참조 음성 후보들 (있는 것만 사용)
analyst_ref_wavs = [
    TTS_REF_DIR / "analyst_pakchanho_prompt_1.wav",
    TTS_REF_DIR / "analyst_pakchanho_prompt_2.wav",
    # TTS_REF_DIR / "analyst_pakchanho_prompt_3.wav",
]

print("LLM CSV:", llm_csv, "| exists:", llm_csv.exists())
for p in caster_ref_wavs:
    print("caster_ref cand:", p, "| exists:", p.exists())
for p in analyst_ref_wavs:
    print("analyst_ref cand:", p, "| exists:", p.exists())

out_csv = run_tts_batch_via_api(
    tts_csv_path=llm_csv,
    caster_ref_wavs=caster_ref_wavs,     # 리스트 통째로 전달
    analyst_ref_wavs=analyst_ref_wavs,   # 리스트 통째로 전달
    api_url="http://127.0.0.1:8080/v1/tts",
    # max_rows=5,  # 필요하면 일부만
)

out_csv

# 260 Row 캐스터 및 해설위원 TTS 생성 450초

LLM CSV: /workspace/baseball_pipeline/data/llm_outputs/clip_test4.tts_phrases.llm_kanana.csv | exists: True
caster_ref cand: /workspace/baseball_pipeline/data/tts_refs/caster_prompt_1.wav | exists: True
analyst_ref cand: /workspace/baseball_pipeline/data/tts_refs/analyst_pakchanho_prompt_1.wav | exists: True
analyst_ref cand: /workspace/baseball_pipeline/data/tts_refs/analyst_pakchanho_prompt_2.wav | exists: True
[TTS_API] 입력 CSV: /workspace/baseball_pipeline/data/llm_outputs/clip_test4.tts_phrases.llm_kanana.csv
[TTS_API] video_stem: clip_test4
[TTS_API] 출력 디렉토리: /workspace/baseball_pipeline/data/tts_audio/clip_test4
[TTS_API] loaded caster ref: /workspace/baseball_pipeline/data/tts_refs/caster_prompt_1.wav
[TTS_API] loaded analyst ref: /workspace/baseball_pipeline/data/tts_refs/analyst_pakchanho_prompt_1.wav
[TTS_API] loaded analyst ref: /workspace/baseball_pipeline/data/tts_refs/analyst_pakchanho_prompt_2.wav
[TTS_API] (0) role=caster voice=caster utt=clip_1_1
[TTS_API] Text: 오늘 두 차

In [5]:
from pathlib import Path
from src import DATA_DIR
from src.audio_concat_test import build_sequential_tts_video

INPUT_VIDEO_DIR   = DATA_DIR / "input_videos"
LLM_OUT_DIR       = DATA_DIR / "llm_outputs"
TTS_AUDIO_ROOT    = DATA_DIR / "tts_audio"
OUTPUT_VIDEO_DIR  = DATA_DIR / "output_videos"

VIDEO_NAME = "clip_test4.mp4"
VIDEO_STEM = Path(VIDEO_NAME).stem

llm_csv       = LLM_OUT_DIR / f"{VIDEO_STEM}.tts_phrases.llm_kanana.csv"
tts_audio_dir = TTS_AUDIO_ROOT / VIDEO_STEM
input_video   = INPUT_VIDEO_DIR / VIDEO_NAME

out_wav   = tts_audio_dir / f"{VIDEO_STEM}.concat_test_gap_gain.wav"
out_video = OUTPUT_VIDEO_DIR / f"{VIDEO_STEM}.concat_test_gap_gain.mp4"

role_gain = {
    "caster": 0.8,
    "analyst": -1.0,
}

final_video = build_sequential_tts_video(
    llm_csv_path=llm_csv,
    tts_audio_dir=tts_audio_dir,
    input_video_path=input_video,
    out_wav_path=out_wav,
    out_video_path=out_video,
    mute_original=True,
    gap_ms_between_roles=50,
    role_gain_db=role_gain,   # 🔹 이제 에러 안 나야 정상
)

final_video


[CONCAT] apply gain +0.80 dB for role=caster (utt=clip_1_1)
[CONCAT] append utt=clip_1_1 role=caster dur=5.76s
[CONCAT] insert gap 50ms between caster -> analyst
[CONCAT] apply gain -1.00 dB for role=analyst (utt=clip_2_1)
[CONCAT] append utt=clip_2_1 role=analyst dur=37.71s
[CONCAT] insert gap 50ms between analyst -> caster
[CONCAT] apply gain +0.80 dB for role=caster (utt=clip_3_1)
[CONCAT] append utt=clip_3_1 role=caster dur=7.66s
[CONCAT] saved full TTS wav: /workspace/baseball_pipeline/data/tts_audio/clip_test4/clip_test4.concat_test_gap_gain.wav
[CUT] video_ms=34859ms (34.86s), audio_ms=51230ms (51.23s)
[CUT] trimmed audio to video length: /workspace/baseball_pipeline/data/tts_audio/clip_test4/clip_test4.concat_test_gap_gain.wav
[MUX(CONCAT)] CMD: ffmpeg -y -i /workspace/baseball_pipeline/data/input_videos/clip_test4.mp4 -i /workspace/baseball_pipeline/data/tts_audio/clip_test4/clip_test4.concat_test_gap_gain.wav -map 0:v:0 -map 1:a:0 -c:v copy -c:a aac -shortest /workspace/baseb

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

[MUX(CONCAT)] saved video: /workspace/baseball_pipeline/data/output_videos/clip_test4.concat_test_gap_gain.mp4


[out#0/mp4 @ 0x5a36c5650940] video:89476kB audio:298kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.033798%
size=   89804kB time=00:00:34.85 bitrate=21107.9kbits/s speed=  54x    
[aac @ 0x5a36c566cb80] Qavg: 303.891


PosixPath('/workspace/baseball_pipeline/data/output_videos/clip_test4.concat_test_gap_gain.mp4')

In [4]:
from pathlib import Path
from src import DATA_DIR
from src.audio_concat_test import build_sequential_tts_video

INPUT_VIDEO_DIR   = DATA_DIR / "input_videos"
LLM_OUT_DIR       = DATA_DIR / "llm_outputs"
TTS_AUDIO_ROOT    = DATA_DIR / "tts_audio"
OUTPUT_VIDEO_DIR  = DATA_DIR / "output_videos"

VIDEO_NAME = "clip_test4.mp4"
VIDEO_STEM = Path(VIDEO_NAME).stem

llm_csv       = LLM_OUT_DIR / f"{VIDEO_STEM}.tts_phrases.llm_kanana.csv"
tts_audio_dir = TTS_AUDIO_ROOT / VIDEO_STEM
input_video   = INPUT_VIDEO_DIR / VIDEO_NAME

out_wav   = tts_audio_dir / f"{VIDEO_STEM}.concat_test_gap.wav"
out_video = OUTPUT_VIDEO_DIR / f"{VIDEO_STEM}.concat_test_gap.mp4"

final_video = build_sequential_tts_video(
    llm_csv_path=llm_csv,
    tts_audio_dir=tts_audio_dir,
    input_video_path=input_video,
    out_wav_path=out_wav,
    out_video_path=out_video,
    mute_original=True,
    gap_ms_between_roles=50,   # 🔹 캐스터↔해설위원 바뀔 때 0.15초 침묵
)

final_video


[CONCAT] append utt=clip_1_1 role=caster dur=5.48s
[CONCAT] insert gap 50ms between caster -> analyst
[CONCAT] append utt=clip_2_1 role=analyst dur=34.92s
[CONCAT] insert gap 50ms between analyst -> caster
[CONCAT] append utt=clip_3_1 role=caster dur=7.25s
[CONCAT] saved full TTS wav: /workspace/baseball_pipeline/data/tts_audio/clip_test4/clip_test4.concat_test_gap.wav
[CUT] video_ms=34859ms (34.86s), audio_ms=47747ms (47.75s)
[CUT] trimmed audio to video length: /workspace/baseball_pipeline/data/tts_audio/clip_test4/clip_test4.concat_test_gap.wav
[MUX(CONCAT)] CMD: ffmpeg -y -i /workspace/baseball_pipeline/data/input_videos/clip_test4.mp4 -i /workspace/baseball_pipeline/data/tts_audio/clip_test4/clip_test4.concat_test_gap.wav -map 0:v:0 -map 1:a:0 -c:v copy -c:a aac -shortest /workspace/baseball_pipeline/data/output_videos/clip_test4.concat_test_gap.mp4


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

[MUX(CONCAT)] saved video: /workspace/baseball_pipeline/data/output_videos/clip_test4.concat_test_gap.mp4


[out#0/mp4 @ 0x5cc62269f680] video:89476kB audio:297kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.033798%
size=   89803kB time=00:00:34.85 bitrate=21107.7kbits/s speed=59.9x    
[aac @ 0x5cc62272ccc0] Qavg: 293.798


PosixPath('/workspace/baseball_pipeline/data/output_videos/clip_test4.concat_test_gap.mp4')